<a href="https://colab.research.google.com/github/leochartrand/IFT615/blob/main/CNN/CNN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Cette démonstration est inspirée du cours "CS50’s Introduction to Artificial Intelligence with Python" donné à l'Université d'Harvard.*

https://cs50.harvard.edu/ai/2024/

https://creativecommons.org/licenses/by-nc-sa/4.0/





In [24]:
import numpy as np
from PIL import Image, ImageFilter
from skimage.io import imshow
import matplotlib.pyplot as plt
from IPython.display import display
import torch